In [ ]:

from typing import Optional, Literal
from pydantic import BaseModel, Field
from langchain.output_parsers import StructuredOutputParser,ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

In [21]:
prompt = PromptTemplate(
    input_variables=["user_input"],  # only user_input will be passed dynamically
    partial_variables={"format_instructions": parser.get_format_instructions()},
    template="""
You are an expert writing evaluator. Your task is to critically evaluate the following essay:

Essay:
\"\"\"
{user_input}
\"\"\"

Evaluate the essay across the following dimensions:

1. **Content & Ideas**: Are the ideas clear, original, and well-developed?
2. **Organization**: Is the structure logical, with a strong introduction, body, and conclusion?
3. **Language & Style**: Is the language formal, clear, and appropriate for the topic?
4. **Grammar & Mechanics**: Are there any grammatical, punctuation, or spelling errors?


Give rating and feeback in the following format:
{format_instructions}
"""
)


In [ ]:
"model we used = mistralai/Mixtral-8x7B-Instruct-v0.1 "

In [13]:
response_schemas = [
    ResponseSchema(name="Rate", description="Rate the essay in the scale of 1 to 10"),
    ResponseSchema(name="feedback", description="Give ur feedback as a teacher for the essay")
]

In [14]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Rate": string  // Rate the essay in the scale of 1 to 10
	"feedback": string  // Give ur feedback as a teacher for the essay
}
```


In [22]:
chain = prompt | model | parser

In [24]:
essay_text = """
Technology has improved communication, education, and healthcare. However, excessive screen time can harm mental health.
It’s important to balance usage and ensure children are educated about responsible tech use.
"""

result = chain.invoke({"user_input": essay_text})
result

{'Rate': '7',
 'feedback': "The essay presents a clear and well-understood view of the advantages and disadvantages of technology. The ideas are straightforward and concise, but lack originality and in-depth analysis.\n\nThe organization of the essay is logical and coherent. The introduction sets the context and presents the main ideas, the body develops the topic further, and the conclusion summarizes the main points.\n\nThe language is formal and clear, suitable for the topic. However, there is room for improvement in the style, as the essay lacks variety and engagement.\n\nThere are a few minor grammatical, punctuation, and spelling errors. It's recommended to proofread the text carefully to ensure correctness and clarity."}

In [25]:
result['Rate']

'7'